In [2]:
import rx_4n_2k.model as model
import rx_4n_2k.__init__ as init
import os

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import pandas as pd
from tensorflow.keras.optimizers import Adam


In [83]:
k = 2
n = 2

learning_rate = 1e-2
n_epochs = 10

weight_path = 'rx_4n_2k\weights.h5'
data_path = r'data.xlsx'

best_model_name = 'best_model.h5'

In [69]:
def load_model(load_weights=True, debug=False):
    m = model.create_model()
    if load_weights:
        loadWeights(m, debug=debug)
    return m

## Utility functions:

import tensorflow as tf
import h5py

def loadWeights(model, filename=weight_path, debug=False):
    with h5py.File(filename, 'r') as f:
        # Every layer is an h5 group. Ignore non-groups (such as /0)
        for g in f:
            if isinstance(f[g], h5py.Group):
                group = f[g]
                layerName = group.attrs['Name']
                numVars = int(group.attrs['NumVars'])
                if debug:
                    print("layerName:", layerName)
                    print("    numVars:", numVars)
                # Find the layer index from its namevar
                layerIdx = layerNum(model, layerName)
                layer = model.layers[layerIdx]
                if debug:
                    print("    layerIdx=", layerIdx)
                # Every weight is an h5 dataset in the layer group. Read the weights 
                # into a list in the correct order
                weightList = [0]*numVars
                for d in group:
                    dataset = group[d]
                    varName = dataset.attrs['Name']
                    shp     = intList(dataset.attrs['Shape'])
                    weightNum = int(dataset.attrs['WeightNum'])
                    # Read the weight and put it into the right position in the list
                    if debug:
                        print("    varName:", varName)
                        print("        shp:", shp)
                        print("        weightNum:", weightNum)
                    weightList[weightNum] = tf.constant(dataset[()], shape=shp)
                # Assign the weights into the layer
                for w in range(numVars):
                    if debug:
                        print("Copying variable of shape:")
                        print(weightList[w].shape)
                    layer.variables[w].assign(weightList[w])
                    if debug:
                        print("Assignment successful.")
                        print("Set variable value:")
                        print(layer.variables[w])
                # Finalize layer state
                if hasattr(layer, 'finalize_state'):
                    layer.finalize_state()

def layerNum(model, layerName):
    # Returns the index to the layer
    layers = model.layers
    for i in range(len(layers)):
        if layerName==layers[i].name:
            return i
    print("")
    print("WEIGHT LOADING FAILED. MODEL DOES NOT CONTAIN LAYER WITH NAME: ", layerName)
    print("")
    return -1

def intList(myList): 
    # Converts a list of numbers into a list of ints.
    return list(map(int, myList))

In [70]:
def decimal_to_binary_array(decimal_array,k):

  binary_array = np.zeros((len(decimal_array), k), dtype=np.float32)

  # Use bitwise AND operation with powers of 2 to get individual bits
  for i, num in enumerate(decimal_array):
    for j in range(k):
      binary_array[i, k-j-1] = (num & (2**j)) != 0
  return binary_array


In [87]:
decoder = load_model(load_weights=True, debug=False)

# Getting data and setting up inputs and outputs to the model
df = pd.read_excel(data_path)
inp_stream = np.array([complex(i) for i in df.iloc[:, 0]])
inp_stream = np.concatenate((np.reshape(np.real(inp_stream),(-1,1)),np.reshape(np.imag(inp_stream),(-1,1))),axis=1)
inp_stream = np.reshape(inp_stream,(-1,2*n))  # convert to a one demensional array
out_stream = np.array([int(i) for i in df.iloc[:, 1]])
out_stream = decimal_to_binary_array(out_stream,k)
# Spliting
train_x,val_x = np.split(inp_stream,[int(0.75 * len(inp_stream))])
train_y,val_y = np.split(out_stream,[int(0.75 * len(out_stream))])
print(out_stream)
print(len(out_stream))
print(train_x,train_y)
print(val_x,val_y)

ValueError: cannot convert float NaN to integer

In [86]:
# Train decoder
decoder.compile(optimizer=Adam(learning_rate=learning_rate),loss="binary_crossentropy",matrics = ['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience =n_epochs//10)
mc = ModelCheckpoint(best_model_name,monitor='val_accuracy',mode='max',verbose=1,save_best_only=True)
history = decoder.fit(train_x,train_y,validation_data = (val_x,val_y),epochs = n_epochs,verbose=2)

ValueError: Data cardinality is ambiguous:
  x sizes: 6
  y sizes: 12
Make sure all arrays contain the same number of samples.